<a href="https://colab.research.google.com/github/zaneddu/Flowise/blob/main/Separatore_buste_paga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
"""
Script CORRETTO per separare buste paga PDF
Versione per Google Colab con interfaccia
"""

import re
from pathlib import Path
import zipfile
from io import BytesIO

try:
    from pypdf import PdfReader, PdfWriter
except ImportError:
    print("📦 Installazione pypdf...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'pypdf', '-q'])
    from pypdf import PdfReader, PdfWriter

from google.colab import files
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

def extract_employee_info(page_text):
    """Estrae cognome, nome e mese dalla pagina - PATTERN CORRETTO"""

    # Cerca il mese
    month_pattern = r'(Gennaio|Febbraio|Marzo|Aprile|Maggio|Giugno|Luglio|Agosto|Settembre|Ottobre|Novembre|Dicembre)\s+(\d{4})'
    month_match = re.search(month_pattern, page_text)

    if not month_match:
        return None, None, None, None

    month = month_match.group(1)
    year = month_match.group(2)

    # PATTERN CORRETTO per trovare "13 DIONISIO LUANA"
    # Cerca: numero (1-3 cifre) + spazio + COGNOME + spazio + NOME
    # Ma solo se è su una riga da sola o seguita da newline
    lines = page_text.split('\n')

    for line in lines:
        line = line.strip()
        # Pattern: numero seguito da nome e cognome in maiuscolo
        # Esempi: "13 DIONISIO LUANA", "99 NANNI EFISIO", "1 MOIMAS VALENTINA"
        name_pattern = r'^(\d{1,3})\s+([A-Z]+)\s+([A-Z]+(?:\s+[A-Z]+)*)$'
        match = re.match(name_pattern, line)

        if match:
            code = match.group(1)
            surname = match.group(2)
            name = match.group(3)

            # Verifica che non siano keywords di sistema
            system_keywords = ['CODICE', 'COGNOME', 'NOME', 'SEDE', 'MESE', 'DATA']
            if surname not in system_keywords and name not in system_keywords:
                return surname, name, month, year

    return None, None, None, None

def split_payslips(pdf_bytes, output_folder='buste_paga_separate'):
    """Separa le buste paga per dipendente"""

    # Crea cartella di output
    output_path = Path(output_folder)
    output_path.mkdir(exist_ok=True)

    # Leggi il PDF dai bytes
    reader = PdfReader(BytesIO(pdf_bytes))
    print(f"📄 PDF caricato: {len(reader.pages)} pagine totali\n")

    # Dizionario per raggruppare pagine per dipendente
    employees = {}
    current_employee = None

    # Analizza ogni pagina
    print("🔍 Analisi pagine in corso...\n")
    for page_num, page in enumerate(reader.pages, 1):
        text = page.extract_text()
        surname, name, month, year = extract_employee_info(text)

        if surname and name:
            # Nuova busta paga trovata
            current_employee = f"{surname}"
            if current_employee not in employees:
                employees[current_employee] = {
                    'pages': [],
                    'month': month,
                    'year': year,
                    'full_name': f"{surname} {name}"
                }
            employees[current_employee]['pages'].append(page)
            print(f"✓ Pagina {page_num}: {surname} {name} - {month} {year}")
        elif current_employee:
            # Pagina di continuazione
            employees[current_employee]['pages'].append(page)
            print(f"  → Pagina {page_num}: Continuazione di {current_employee}")
        else:
            print(f"⚠️  Pagina {page_num}: Dipendente non identificato (pagina ignorata)")

    if not employees:
        raise Exception("Nessun dipendente trovato nel PDF! Verifica il formato.")

    # Crea PDF separati
    print(f"\n📝 Creazione {len(employees)} buste paga separate...\n")
    created_files = []

    for emp_surname, emp_data in employees.items():
        writer = PdfWriter()
        for page in emp_data['pages']:
            writer.add_page(page)

        filename = f"{emp_surname}_{emp_data['month']}_{emp_data['year']}.pdf"
        output_file = output_path / filename

        with open(output_file, 'wb') as f:
            writer.write(f)

        created_files.append(filename)
        print(f"✓ {filename} ({len(emp_data['pages'])} pag.) - {emp_data['full_name']}")

    print(f"\n✅ Completato! {len(employees)} buste paga create")
    return output_path, created_files

def create_zip(folder_path):
    """Crea un file ZIP con tutte le buste paga"""
    zip_buffer = BytesIO()

    pdf_files = list(folder_path.glob('*.pdf'))
    print(f"\n📦 Creazione ZIP con {len(pdf_files)} file...\n")

    with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for pdf_file in pdf_files:
            zip_file.write(pdf_file, pdf_file.name)
            print(f"  ✓ Aggiunto: {pdf_file.name}")

    zip_buffer.seek(0)

    # Verifica contenuto
    with zipfile.ZipFile(BytesIO(zip_buffer.getvalue()), 'r') as verify:
        files_in_zip = verify.namelist()
        print(f"\n🔍 Verifica: {len(files_in_zip)} file nello ZIP")
        if len(files_in_zip) != len(pdf_files):
            print("⚠️  ATTENZIONE: Il numero di file non corrisponde!")

    return zip_buffer

def main():
    """Funzione principale con interfaccia"""

    # Stile CSS
    display(HTML("""
    <style>
        .big-title {
            font-size: 28px;
            font-weight: bold;
            color: #1976D2;
            margin-bottom: 20px;
            text-align: center;
        }
        .instructions {
            background-color: #E3F2FD;
            padding: 15px;
            border-radius: 8px;
            margin-bottom: 20px;
            font-size: 14px;
        }
        .success-box {
            background-color: #C8E6C9;
            padding: 15px;
            border-radius: 8px;
            margin-top: 20px;
            font-size: 14px;
        }
    </style>
    """))

    # Titolo
    display(HTML('<div class="big-title">📋 SEPARATORE BUSTE PAGA (VERSIONE CORRETTA)</div>'))

    # Istruzioni
    display(HTML("""
    <div class="instructions">
        <b>📖 Istruzioni:</b><br>
        1️⃣ Clicca sul pulsante "Carica PDF" qui sotto<br>
        2️⃣ Seleziona il file PDF con tutte le buste paga<br>
        3️⃣ Aspetta che lo script finisca di processare<br>
        4️⃣ Clicca sul pulsante "Scarica ZIP" per scaricare tutti i file separati
    </div>
    """))

    # Area output
    output_area = widgets.Output()

    # Variabili globali
    zip_data = {'buffer': None}

    def on_upload_click(b):
        with output_area:
            clear_output()
            print("📂 Seleziona il file PDF...\n")

            # Carica il file
            uploaded = files.upload()

            if not uploaded:
                print("❌ Nessun file caricato")
                return

            # Prendi il primo file caricato
            filename = list(uploaded.keys())[0]
            pdf_bytes = uploaded[filename]

            print(f"📥 File caricato: {filename}")
            print(f"📊 Dimensione: {len(pdf_bytes) / 1024:.1f} KB\n")
            print("="*60 + "\n")

            try:
                # Processa il PDF
                output_folder, created_files = split_payslips(pdf_bytes)

                # Crea ZIP
                zip_buffer = create_zip(output_folder)
                zip_data['buffer'] = zip_buffer

                # Mostra successo
                display(HTML(f"""
                <div class="success-box">
                    <b>✅ OPERAZIONE COMPLETATA!</b><br><br>
                    📊 <b>Buste paga create:</b> {len(created_files)}<br>
                    📁 <b>File nello ZIP:</b><br>
                    {'<br>'.join(['   • ' + f for f in created_files])}<br><br>
                    👇 <b>Clicca sul pulsante "Scarica ZIP" qui sotto</b>
                </div>
                """))

                # Abilita il pulsante download
                download_button.disabled = False

            except Exception as e:
                print(f"\n❌ ERRORE: {str(e)}\n")
                import traceback
                traceback.print_exc()

    def on_download_click(b):
        with output_area:
            if zip_data['buffer']:
                print("\n💾 Download in corso...")
                with open('buste_paga_separate.zip', 'wb') as f:
                    f.write(zip_data['buffer'].getvalue())
                files.download('buste_paga_separate.zip')
                print("✅ Download completato!")
            else:
                print("❌ Nessun file da scaricare. Carica prima un PDF!")

    # Pulsanti
    upload_button = widgets.Button(
        description='📤 Carica PDF',
        button_style='primary',
        layout=widgets.Layout(width='200px', height='50px'),
        style={'font_weight': 'bold'}
    )

    download_button = widgets.Button(
        description='💾 Scarica ZIP',
        button_style='success',
        layout=widgets.Layout(width='200px', height='50px'),
        style={'font_weight': 'bold'},
        disabled=True
    )

    upload_button.on_click(on_upload_click)
    download_button.on_click(on_download_click)

    # Layout
    buttons = widgets.HBox([upload_button, download_button],
                          layout=widgets.Layout(justify_content='center', margin='20px'))

    display(buttons)
    display(output_area)

# Avvio automatico
if __name__ == "__main__":
    main()

Output()